# 12.1 Your First Agent

In [6]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

prompt = "Tell me Total Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

llm.invoke(prompt)

AIMessage(content='The total cost is $4,293.38.')

위 계산을 하면 실제로 값은 $5273.38이 나온다. 그러나 지금 llm은 실수를 하고 있다. 즉, LLM이 틀렸다. 저건 진짜 Cost값이 아니다.

분명히 LLM은 산수를 할 수 없다. 계산기에는 AI가 존재하지 않는다. 하지만 계산기가 이걸 더 잘 해낼 것이다. 왜냐하면 LLM은 모든 종류의 연산을 실행하는 그런게 아니다. LLM은 text를 완성하는 것이다. 그냥 다음 token은 무엇인지 통계적으로 추측하는 것이다.

그래서 우리는 무엇을 해야할까? 이 error를 fix하기 위해 우리는 agent를 만들고, 또 agent를 위한 tool을 작성해줄 것이다. 따라서 agent는 이 계산을 수행하기 위해 tool을 선택할 것이다. 그냥 결과를 예측하다가 실수를 하는 대신 말이다.

In [9]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool
# 우리 agent가 실제로 사용할 수 있는 tool로 변환하기 위한 클래스
# 이 StructuredTool은 우리의 tool이 여러개의 input(arguments)를 가질때 사용한다.
# 실제 이 코드에서 plus라는 함수는 두개의 argument를 받는다.
# 참고로 이전에 오래된 version의 langchain에서는 단 하나의 input만 가질 수 있었다.

def sum_all_numbers(a, b):
    return a + b

agent = initialize_agent(
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    # 어떤 종류의 agent로 initialize(초기화)할지 정하기 위한 코드
    # 여기서 "STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION"는 여러개의 input을 가진 tool들을 invoke할 수 있다는 뜻이다.
    # 이름을 풀어보면, STRUCTURED라는 뜻은 tool이 한개보다 많은 input을 가질 수 있다는 의미이다.
    # 그리고 CHAT이라고 쓰여진 이유는 Chat Model을 위해 최적화된 agent라는 뜻이다.
    # ZERO SHOT REACT라는 뜻은 특정 type의 agent라는 뜻이다. 나중에 살펴 볼 것이다.
    
    verbose=True,
    # 우리에게 agent가 하는 작업의 모든 과정 및 추론을 보여주기 위한 코드 줄.

    tools=[
        StructuredTool.from_function(
            func=sum_all_numbers, 
            # 우리가 어떤 함수를 tool로 사용할 건지를 전달해줘야 한다. 이때 함수명만 전달해주면 된다.
            # 함수 호출이나 문자열로 전달하면 안된다.

            name="Total Sum Calculator", # 이 tool에게 이름을 부여하기 위해 작성함

            description="Use ths to perform sums of two numbers. This tool take two arguments, both should be numbers. Till finish.",
            # 이제 이 함수를 설명해주는 description을 작성한다.
            # 함수가 어떤 역할을 하는지, 어떨때 이 tool을 사용해야 하는지를 LLM에게 설명하기 위함이다.
        ),
    ],
)

agent.invoke(prompt)




> Entering new AgentExecutor chain...
Thought: I can use the Total Sum Calculator tool to find the total cost of all the given amounts.

Action:
```
{
  "action": "Total Sum Calculator",
  "action_input": {"a": 355.39, "b": 924.87}
}
```


Observation: 1280.26
Thought:I have calculated the sum of the first two amounts correctly. Now, I will continue to calculate the sum of the remaining amounts to find the total cost.
Action:
```
{
  "action": "Total Sum Calculator",
  "action_input": {"a": 1280.26, "b": 721.2}
}
```
Observation: 2001.46
Thought:I have calculated the sum of the first three amounts correctly. Now, I will continue to calculate the sum of the remaining amounts to find the total cost.

Action:
```
{
  "action": "Total Sum Calculator",
  "action_input": {"a": 2001.46, "b": 1940.29}
}
```
Observation: 3941.75
Thought:I have calculated the sum of the first four amounts correctly. Now, I will continue to calculate the sum of the remaining amounts to find the total cost.

Act

{'input': 'Tell me Total Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}

코드의 설명과 이 agent가 어떻게 수행되고 작동되는지는 다음에 설명